In [11]:
import sys
import os
import pprint # pretty print module: 데이터를 보기 좋게 출력할 때 사용
import time
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torchvision

from torch.utils.data.sampler import Sampler
from torch.utils.data import DataLoader
from datasets.CustomVOCDetection import CustomVOCDetection
from roi_data_layer.roi_data_layer import combined_roidb
from roi_data_layer.roibatchLoader import roibatchLoader

In [2]:
# define arguments
n_epochs = 20
save_dir = 'save'
is_ohem = True
batch_size = 1
lr = 0.001
lr_decay_step = 5
lr_decay_gamma = 0.1

In [3]:
class sampler(Sampler):
    def __init__(self, train_size, batch_size):
        self.train_size = train_size
        self.batch_size = batch_size
        self.num_batch = int(train_size / batch_size)
        self.range = torch.arange(0, batch_size).view(1, batch_size).long()
        self.leftover_flag = False
        if train_size % batch_size:
            self.leftover = torch.arange(self.num_per_batch*batch_size, train_size).long()
            self.leftover_flag = True

    def __iter__(self):
        rand_batch_idx = torch.randperm(self.num_batch).view(-1, 1) * self.batch_size
        self.rand_batch_idx = rand_batch_idx.expand(self.num_batch, self.batch_size) + self.range

        self.rand_idx = self.rand_num.view(-1)

        if self.leftover_flag:
            self.rand_idx = torch.cat((self.rand_idx, self.leftover), 0)

        return iter(self.rand_idx)
    
    def __len__(self):
        return self.train_size

In [4]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'gpu'
elif torch.backends.mps.is_available():
    device = 'mps'
print(f"{device} is being used.")

cpu is being used.


In [5]:
pascal_voc_dataset = CustomVOCDetection(
    root=os.path.abspath(os.path.join(os.getcwd(), 'datasets')),
    year='2012',
    image_set='train',
    download=False,
)
roidb, ratio_list, ratio_index = combined_roidb(pascal_voc_dataset)
train_size = len(roidb)

print("{:d} roidb entries".format(train_size))

pascal_voc_2012_train gt roidb loaded from /workspaces/R-FCN_with_Pytorch/datasets/cache/pascal_voc_2012_train_gt_roidb.pkl
Preparing training data...
Done
before filtering, there are 5717 images...
after filtering, there are 5717 images...
5717 roidb entries


In [6]:
pprint.pprint(roidb[0])

{'areas': array([139946.,  16368.], dtype=float32),
 'boxes': array([[ 52,  86, 470, 419],
       [157,  43, 288, 166]], dtype=uint16),
 'flipped': False,
 'gt_classes': array([13, 15], dtype=int32),
 'gt_difficult': array([0, 0], dtype=int32),
 'gt_ious': <2x21 sparse matrix of type '<class 'numpy.float32'>'
	with 2 stored elements in Compressed Sparse Row format>,
 'height': 442,
 'image': '/workspaces/R-FCN_with_Pytorch/datasets/VOCdevkit/VOC2012/JPEGImages/2008_000008.jpg',
 'image_id': 0,
 'max_classes': array([13, 15]),
 'max_ious': array([1., 1.], dtype=float32),
 'need_crop': 0,
 'width': 500}


In [8]:
output_dir = os.path.join(os.getcwd(), 'save')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [9]:
sampler_batch = sampler(train_size, batch_size)

dataset = roibatchLoader(roidb, ratio_list, ratio_index, batch_size, 
                         pascal_voc_dataset.num_classes, training=True)

dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler_batch)

In [10]:
# initialize tensor holder 
image_data = torch.FloatTensor(1)
image_info = torch.FloatTensor(1)
num_boxes = torch.LongTensor(1)
gt_boxes = torch.FloatTensor(1)

# ship to cuda
if device == 'gpu':
    image_data = image_data.cuda()
    image_info = image_info.cuda()
    num_boxes = num_boxes.cuda()
    gt_boxes = gt_boxes.cuda()

In [12]:
base_model = torchvision.models.resnet101(weights="IMAGENET1K_V1")

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /home/codespace/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100.0%
